# Gather TBA Data

Read an ini file regarding authorization, match and last update time.

Contact TBA and download information regarding match results for all matches, OPR for all teams, and overall insights for the event.
Save all of the information to a CSV for scouting.

In [12]:
import configparser
import glob
import logging

import numpy as np
import pandas as pd

import re
import sys
import tbaapiv3client
from tbaapiv3client.rest import ApiException

logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logger = logging.getLogger('Gather TBA')

# Setup TBA connection
# make call to TBA for data
config = configparser.ConfigParser()
config.read(r'tba.ini')
event_key = config['TBA']['event_key']
configuration = tbaapiv3client.Configuration(
    host = config['TBA']['tbahost'] ,
    api_key = { 'X-TBA-Auth-Key': config['TBA']['auth_key'] }
)

In [13]:
# OPR -> Offensive Power Rating
# DPR -> Defensive Power Rating
# CCWM -> “calculated contribution to win margin”
df = pd.DataFrame( columns = ["team","ccwms","dprs","oprs"])

with tbaapiv3client.ApiClient(configuration) as api_client:
    api_instance = tbaapiv3client.EventApi(api_client)    
    try:
        api_response = api_instance.get_event_op_rs(event_key ) # TODO: if modified since
    except ApiException as e:
        log.error("Calling EventApi->get_event_op_rs: %s" % e)

In [14]:
result_data = api_response.to_dict()
ccwms = result_data['ccwms']
oprs = result_data['oprs']
dprs = result_data['dprs']

for key in oprs.keys() :
    team = re.sub('frc','', key )
    if ( re.match('^[0-9]+$',team) is None ):
        logger.error ( key + " does not match pattern expected of frc[0-9]* ")
        raise SystemExit("Need to stop")
    df.loc[len(df)] = {'team': team, 'ccwms': ccwms[key], 'oprs': oprs[key], 'dprs': dprs[key]}
df.to_csv('tba_oprs.csv')

In [15]:
# get_event_matches will get those who are finished, as well as matches that are just scheduled.
# post_result_time will be "None" for matches that have not been played, yet.

with tbaapiv3client.ApiClient(configuration) as api_client:
    api_instance = tbaapiv3client.EventApi(api_client)    
    try:
        api_response = api_instance.get_event_matches(event_key ) # TODO: if modified since
    except ApiException as e:
        log.error("Calling EventApi->get_event_matches: %s" % e )

In [16]:
df = pd.DataFrame(columns=["match_number", "post_result_time",
                           "red1","red2","red3","red1_taxi","red2_taxi","red3_taxi",
                           "red1_climb","red2_climb","red3_climb","red_points",
                           "blue1","blue2","blue3","blue1_taxi","blue2_taxi","blue3_taxi",
                           "blue1_climb","blue2_climb","blue3_climb","blue_points"])
#print(api_response)
for x in api_response:
   if ( x.comp_level == "qm" ):
      #only gathering scouting information for the qualifying matches.
      if ( x.set_number != 1 ):
        logger.warning("Set Number non zero, not sure what to do with this.  Match: %d" % x.match_number)
        logger.debug(x)
      ra=x.alliances.to_dict()['red']
      ba=x.alliances.to_dict()['blue']
      red=x.score_breakdown['red']
      blue=x.score_breakdown['blue']
      
      red1 = re.sub('frc','',ra['team_keys'][0])
      red2 = re.sub('frc','',ra['team_keys'][1])
      red3 = re.sub('frc','',ra['team_keys'][2])
      blue1 = re.sub('frc','',ba['team_keys'][0])
      blue2 = re.sub('frc','',ba['team_keys'][1])
      blue3 = re.sub('frc','',ba['team_keys'][2])
   
      if ( ra['dq_team_keys'] or ba['dq_team_keys'] ):
         logger.warning('A match had dq teams.  Not sure what to do with this.  Match: %d' % x.match_number)
         logger.debug(ra)
         logger.debug(ba)
      df.loc[len(df)] = {'match_number' : x.match_number, 'post_result_time': x.post_result_time,
                         'red1': red1, 'red2': red2, 'red3': red3,
                         'red1_taxi': red['taxiRobot1'], 'red2_taxi': red['taxiRobot2'], 'red3_taxi': red['taxiRobot3'], 
                         'red1_climb': red['endgameRobot1'], 'red2_climb': red['endgameRobot2'], 'red3_climb': red['endgameRobot3'],
                         'red_points': red['totalPoints'],
                         'blue1': blue1, 'blue2': blue2, 'blue3': blue3,
                         'blue1_taxi': blue['taxiRobot1'], 'blue2_taxi': blue['taxiRobot2'], 'blue3_taxi': blue['taxiRobot3'], 
                         'blue1_climb': blue['endgameRobot1'], 'blue2_climb': blue['endgameRobot2'], 'blue3_climb': blue['endgameRobot3'],
                         'blue_points': blue['totalPoints']}

df.to_csv('tba_matches.csv')

DEBUG:Gather TBA:{'score': 62, 'team_keys': ['frc4219', 'frc5986', 'frc2881'], 'surrogate_team_keys': [], 'dq_team_keys': ['frc5986']}
DEBUG:Gather TBA:{'score': 55, 'team_keys': ['frc5572', 'frc8573', 'frc6377'], 'surrogate_team_keys': [], 'dq_team_keys': []}
DEBUG:Gather TBA:{'score': 18, 'team_keys': ['frc8818', 'frc5726', 'frc8879'], 'surrogate_team_keys': [], 'dq_team_keys': []}
DEBUG:Gather TBA:{'score': 36, 'team_keys': ['frc3679', 'frc5572', 'frc5986'], 'surrogate_team_keys': [], 'dq_team_keys': ['frc5986']}
DEBUG:Gather TBA:{'score': 40, 'team_keys': ['frc8573', 'frc7088', 'frc3679'], 'surrogate_team_keys': [], 'dq_team_keys': []}
DEBUG:Gather TBA:{'score': 24, 'team_keys': ['frc5986', 'frc8088', 'frc3545'], 'surrogate_team_keys': [], 'dq_team_keys': ['frc5986']}
DEBUG:Gather TBA:{'score': 19, 'team_keys': ['frc5986', 'frc8879', 'frc2789'], 'surrogate_team_keys': [], 'dq_team_keys': ['frc5986']}
DEBUG:Gather TBA:{'score': 34, 'team_keys': ['frc8610', 'frc8818', 'frc2158'], 'su